In [1]:
import basic_image_operations as basic_ops
import filters
import geometrical_image_operations as geo_ops
import traffic_sign_detection as tsd
import numpy as np

bgr_img = basic_ops.load_image(image_path='traffic_sign_images/stop_01.jpg')
bgr_img = geo_ops.resize_image(image=bgr_img, target_height=bgr_img.shape[0]//8, target_width=bgr_img.shape[1]//8)
for y in range(bgr_img.shape[0]):
    for x in range(bgr_img.shape[1]):
        hsv = tsd.rgb_to_hsv(bgr_img[y, x])
        h, s, v = hsv[0], hsv[1], hsv[2]
        if tsd.is_strong_red(h, s, v):
            continue
        bgr_img[y, x] = np.array([0, 0, 0])

erosion_dilation_img = bgr_img
erosion_dilation_img = filters.dilation(image=erosion_dilation_img, dim=9)
erosion_dilation_img = filters.erosion(image=erosion_dilation_img, dim=9)

red_mask = tsd.get_red_mask(erosion_dilation_img)
blobs = tsd.label_connected_components(red_mask)
boxed_image = tsd.draw_bounding_boxes(image=erosion_dilation_img, blobs=blobs, min_box_area=1000)
basic_ops.show_image(image=boxed_image)

Image loaded from traffic_sign_images/stop_01.jpg


Image displayed
